In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import subprocess

class PMYojana():
    def __init__(self) -> None:
        self.DCR = 33e6 #DCR Program cost per MW
        self.NON_DCR = 26e6 #Non-DCR Program cost per MW
        self.INFLATION = 0.06 #Inflation Rate
        self.YOJANA_LENGTH = 25 #How long is the Yojana
        self.UNITS = 4500 #Units per MW
        self.LAND_INCREASE = 0.05 #how much it increases every two years

    def loan_amount(self, project_size: float, DCR_status: bool, loan_size: float, subsidy_size:float):
        '''
        If the loan_size is a percent, this function simply converts it to an amount.
        For example, if you type in loan_size = 70, this will calculate 70% of the project cost.
        '''
        if loan_size <= 100000:
            if DCR_status:
                return ((self.DCR * project_size - subsidy_size)) * (loan_size/100)
            else:
                return ((self.NON_DCR * project_size - subsidy_size)) * (loan_size/100)
        else:
            return loan_size


    def total_amount(self, bid_rate: float, project_size: float):
        '''
        The absolute return without any cost involved. Calculates the gross number.
        '''
        nominal_per_year = project_size * 4500 * 365 * bid_rate #Return in Rupees/per year
        total_return = [nominal_per_year] * self.YOJANA_LENGTH
        solar_breakdown = [1.0]*25
        solar_breakdown[0] = 0.98
        for i in range(1, len(solar_breakdown)):
            solar_breakdown[i] *= solar_breakdown[i-1]*0.994
        for j in range(len(total_return)):
            total_return[j] *= solar_breakdown[j]
        dictionary = {'Year': np.arange(1, 26), 'Gross Return': total_return}
        absolute_structure = pd.DataFrame.from_dict(dictionary)       

        return absolute_structure


    def emi_payment(self, project_size: float, loan_size: float, pay_emi_in: int, subsidy_size: float, bank_loan_rate = 0.105, DCR_status=True):
        '''
        EMI Payment structure for the specified length.
        '''
        structure = [0.0] * self.YOJANA_LENGTH

        #loan in percent
        loan_size = self.loan_amount(project_size=project_size, DCR_status=DCR_status, loan_size=loan_size, subsidy_size=subsidy_size)

        emi_amount = (loan_size * bank_loan_rate)/(1 - (1 + bank_loan_rate)**(-1*pay_emi_in))

        for i in range(pay_emi_in):
            structure[i] = emi_amount
        
        #Creating a dictionary for exit
        dictionary = {'Year': np.arange(1, 26), 'EMI': structure}
        emi_structure = pd.DataFrame.from_dict(dictionary)

        return emi_structure
    
    def nominal_amount(self, bid_rate: float, project_size: float, loan_size: float, pay_emi_in: int, subsidy_size:float, DCR_status=True, cost_per_bigha_per_month = 3e4, monthly_expenses=5e4, raise_rate=1/2):
        '''
        Nominal amount = Total amount - EMI Payment
        '''
        total_amount = self.total_amount(bid_rate=bid_rate, project_size=project_size)
        emi_pay = self.emi_payment(project_size=project_size, loan_size=loan_size, pay_emi_in=pay_emi_in, subsidy_size=subsidy_size, DCR_status=DCR_status)
        land_cost = self.land_cost(project_size=project_size, cost_per_bigha_per_month=cost_per_bigha_per_month)
        expense_cost = self.expenses(monthly_expenses=monthly_expenses, raise_rate=raise_rate)
        nominal_amount = total_amount
        nominal_amount['Nominal Amount'] = total_amount['Gross Return'] - emi_pay['EMI'] - land_cost['Land Cost'] - expense_cost['Expense Cost']

        return nominal_amount[['Year', 'Nominal Amount']]
    
    def inflation_rate(self, ):
        '''
        Calculates the effects of inflation for 25Y.
        '''
        inflation_adjust = [1.0] * self.YOJANA_LENGTH
        for i in range(len(inflation_adjust)):
            inflation_adjust[i] *= (1 + self.INFLATION)**(i+1)

        dictionary = {'Year': np.arange(1, 26), 'Inflation': inflation_adjust}
        inflation_by_year = pd.DataFrame.from_dict(dictionary)

        return inflation_by_year
    
    def real_amount(self, bid_rate: float, project_size: float, loan_size: float, pay_emi_in: int, subsidy_size: float, DCR_status=True, cost_per_bigha_per_month = 3e4, monthly_expenses=5e4, raise_rate=1/2):
        '''
        Nominal amount realized by the inflation factor.
        '''
        #get Nominal Amounts
        nominal_amount = self.nominal_amount(bid_rate=bid_rate, project_size=project_size, loan_size=loan_size, pay_emi_in=pay_emi_in, subsidy_size=subsidy_size, DCR_status=DCR_status, cost_per_bigha_per_month=cost_per_bigha_per_month, monthly_expenses=monthly_expenses, raise_rate=raise_rate)
        
        #turn it into Real Amounts
        real_amount = nominal_amount

        #inflation
        inflation = self.inflation_rate()

        real_amount['Real Amount'] = nominal_amount['Nominal Amount'] / inflation['Inflation']

        return real_amount[['Year', 'Real Amount']]
    
    def annualized_return(self, bid_rate: float, project_size: float, loan_size: float, pay_emi_in: int, subsidy_size: float, realized=True, DCR_status=True, cost_per_bigha_per_month = 3e4, monthly_expenses=5e4, raise_rate=1/2):
        if realized:
            amount = self.real_amount(bid_rate=bid_rate, project_size=project_size, loan_size=loan_size, subsidy_size=subsidy_size, pay_emi_in=pay_emi_in, cost_per_bigha_per_month = cost_per_bigha_per_month, monthly_expenses=monthly_expenses, raise_rate=raise_rate)
            overall_val = sum(amount['Real Amount'])
        else:
            amount = self.nominal_amount(bid_rate=bid_rate, project_size=project_size, loan_size=loan_size, subsidy_size=subsidy_size, pay_emi_in=pay_emi_in, cost_per_bigha_per_month = cost_per_bigha_per_month,monthly_expenses=monthly_expenses, raise_rate=raise_rate)
            overall_val = sum(amount['Nominal Amount'])
        if DCR_status:
            overall_investment = (project_size * self.DCR) - self.loan_amount(project_size=project_size, subsidy_size=subsidy_size, DCR_status=DCR_status, loan_size=loan_size) - subsidy_size
            if realized:
                print(f'Realized return of {round(overall_val, 0)} INR on {round(overall_investment, 2)} investment.')
                print(f'{round((((overall_val/overall_investment)**(1/25))-1)*100, 2)}% return over {self.INFLATION*100}% inflation over {self.YOJANA_LENGTH} years.')
            else:
                print(f'Nominal return of {round(overall_val, 0)} INR on {round(overall_investment, 2)} investment.')
                print(f'{round((((overall_val/overall_investment)**(1/25))-1)*100, 2)}% return over {self.YOJANA_LENGTH} years.')
        else:
            overall_investment = (project_size * self.NON_DCR) - self.loan_amount(project_size=project_size, subsidy_size=subsidy_size, DCR_status=DCR_status, loan_size=loan_size) - subsidy_size
            if realized:
                print(f'Realized return of {round(overall_val, 0)} INR on {round(overall_investment, 2)} investment.')
                print(f'{round((((overall_val/overall_investment)**(1/25))-1)*100, 2)}% return over {self.INFLATION*100}% inflation over {self.YOJANA_LENGTH} years.')
            else:
                print(f'Nominal return of {round(overall_val, 0)} INR on {round(overall_investment, 2)} investment.')
                print(f'{round((((overall_val/overall_investment)**(1/25))-1)*100, 2)}% return over {self.YOJANA_LENGTH} years.')
            
        
        return 

    def figure_total(self, bid_rate: float, project_size: float, loan_size: float, pay_emi_in: int, subsidy_size: float, realized=True, DCR_status=True, cost_per_bigha_per_month = 3e4, monthly_expenses=5e4, raise_rate=1/2):
        '''
        Draw total, nominal, and emi payments.
        '''
        emi_amount = self.emi_payment(project_size=project_size, loan_size=loan_size, subsidy_size=subsidy_size, pay_emi_in=pay_emi_in)
        nominal_amount = self.nominal_amount(bid_rate=bid_rate, project_size=project_size, loan_size=loan_size, subsidy_size=subsidy_size, pay_emi_in=pay_emi_in, cost_per_bigha_per_month = cost_per_bigha_per_month, monthly_expenses=monthly_expenses, raise_rate=raise_rate)
        real_amount = self.real_amount(bid_rate=bid_rate, project_size=project_size, loan_size=loan_size, subsidy_size=subsidy_size, pay_emi_in=pay_emi_in, cost_per_bigha_per_month = cost_per_bigha_per_month, monthly_expenses=monthly_expenses, raise_rate=raise_rate)

        plt.plot(emi_amount['Year'], emi_amount['EMI']/1e5)
        plt.plot(emi_amount['Year'], nominal_amount['Nominal Amount']/1e5)
        plt.plot(emi_amount['Year'], real_amount['Real Amount']/1e5)

        plt.scatter(emi_amount['Year'], emi_amount['EMI']/1e5)
        plt.scatter(emi_amount['Year'], nominal_amount['Nominal Amount']/1e5)
        plt.scatter(emi_amount['Year'], real_amount['Real Amount']/1e5)

        plt.legend(['EMI', 'Nominal Amount', 'Real Amount'])
        plt.xlabel('Year')
        plt.ylabel('In Lakhs')
        plt.grid(True)

        self.annualized_return(bid_rate=bid_rate, project_size=project_size, loan_size=loan_size, pay_emi_in=pay_emi_in, subsidy_size=subsidy_size, realized=realized, DCR_status=DCR_status, cost_per_bigha_per_month = cost_per_bigha_per_month, monthly_expenses=monthly_expenses, raise_rate=raise_rate)
        return
    
    def _annualized_return(self, bid_rate: float, project_size: float, loan_size: float, pay_emi_in: int, subsidy_size: float, realized: bool, DCR_status: bool, cost_per_bigha_per_month = 3e4, monthly_expenses=5e4, raise_rate=1/2):
        if realized:
            amount = self.real_amount(bid_rate=bid_rate, project_size=project_size, loan_size=loan_size, subsidy_size=subsidy_size, pay_emi_in=pay_emi_in, cost_per_bigha_per_month = cost_per_bigha_per_month, monthly_expenses=monthly_expenses, raise_rate=raise_rate)
            overall_val = sum(amount['Real Amount'])
        else:
            amount = self.nominal_amount(bid_rate=bid_rate, project_size=project_size, loan_size=loan_size, subsidy_size=subsidy_size, pay_emi_in=pay_emi_in, cost_per_bigha_per_month = cost_per_bigha_per_month, monthly_expenses=monthly_expenses, raise_rate=raise_rate)
            overall_val = sum(amount['Nominal Amount'])
        if DCR_status:
            overall_investment = (project_size * self.DCR) - self.loan_amount(project_size=project_size, subsidy_size=subsidy_size, DCR_status=DCR_status, loan_size=loan_size) - subsidy_size
        else:
            overall_investment = (project_size * self.NON_DCR) - self.loan_amount(project_size=project_size, subsidy_size=subsidy_size, DCR_status=DCR_status, loan_size=loan_size) - subsidy_size
        
        return (((overall_val/overall_investment)**(1/25))-1)*100

    def _figure_return(self, bid_rate: float, project_size: float, loan_size: float, pay_emi_in: int, subsidy_size: float, compare: str, realized: bool, DCR_status: bool, cost_per_bigha_per_month = 3e4, monthly_expenses=5e4, raise_rate=1/2):
        if compare == 'bid_rate':
            if type(bid_rate) != list:
                x = np.linspace(0.5*bid_rate, 1.5*bid_rate, 10)
            else:
                x = np.linspace(bid_rate[0], bid_rate[-1], 10)
            #output
            y = []
            for val in x:
                y.append(self._annualized_return(bid_rate=val, project_size=project_size, loan_size=loan_size, pay_emi_in=pay_emi_in, subsidy_size=subsidy_size, realized=realized, DCR_status=DCR_status, cost_per_bigha_per_month = cost_per_bigha_per_month, monthly_expenses=monthly_expenses, raise_rate=raise_rate))
            plt.plot(x, y)
            plt.scatter(x, y)
            plt.grid(True)
            plt.xlabel('Bid Rate')
            if realized:
                plt.ylabel(f'Realized Return over {round(self.INFLATION*100, 2)}% inflation')
            else:
                plt.ylabel('Nominal Return')
        elif compare == 'project_size':
            if type(project_size) != list:
                x = np.linspace(0.5*project_size, 1.5*project_size, 10)
            else:
                x = np.linspace(project_size[0], project_size[-1], 10)
            #output
            y = []
            for val in x:
                y.append(self._annualized_return(bid_rate=bid_rate, project_size=val, loan_size=loan_size, pay_emi_in=pay_emi_in, subsidy_size=subsidy_size, realized=realized, DCR_status=DCR_status, cost_per_bigha_per_month = cost_per_bigha_per_month, monthly_expenses=monthly_expenses, raise_rate=raise_rate))
            plt.plot(x, y)
            plt.scatter(x, y)
            plt.grid(True)
            plt.xlabel('Project Size in MW')
            if realized:
                plt.ylabel(f'Realized Return over {round(self.INFLATION*100, 2)}% inflation')
            else:
                plt.ylabel('Nominal Return')
        elif compare == 'loan_size':
            if type(loan_size) != list:
                x = np.linspace(0.75*loan_size, 1.25*loan_size, 10)
            else:
                x = np.linspace(loan_size[0], loan_size[-1], 10)
            #output
            y = []
            for val in x:
                y.append(self._annualized_return(bid_rate=bid_rate, project_size=project_size, loan_size=val, pay_emi_in=pay_emi_in, subsidy_size=subsidy_size, realized=realized, DCR_status=DCR_status, cost_per_bigha_per_month = cost_per_bigha_per_month, monthly_expenses=monthly_expenses, raise_rate=raise_rate))
            plt.plot(x, y)
            plt.scatter(x, y)
            plt.grid(True)
            plt.xlabel('Loan Amount')
            if realized:
                plt.ylabel(f'Realized Return over {round(self.INFLATION*100, 2)}% inflation')
            else:
                plt.ylabel('Nominal Return')
        elif compare == 'pay_emi_in':
            if type(pay_emi_in) != list:
                x = np.arange(int(0.5*pay_emi_in), min(int(1.5*pay_emi_in) + 1, 16), 1)
            else:
                x = np.arange(pay_emi_in[0], pay_emi_in[-1], 1)
            #output
            y = []
            for val in x:
                y.append(self._annualized_return(bid_rate=bid_rate, project_size=project_size, loan_size=loan_size, pay_emi_in=val, subsidy_size=subsidy_size, realized=realized, DCR_status=DCR_status, cost_per_bigha_per_month = cost_per_bigha_per_month, monthly_expenses=monthly_expenses, raise_rate=raise_rate))
            plt.plot(x, y)
            plt.scatter(x, y)
            plt.grid(True)
            plt.xlabel('EMI Duration')
            if realized:
                plt.ylabel(f'Realized Return over {round(self.INFLATION*100, 2)}% inflation')
            else:
                plt.ylabel('Nominal Return')
        elif compare == 'subsidy_size':
            if type(subsidy_size) != list:
                x = np.linspace(0.5*subsidy_size, 1.5*subsidy_size, 10)
            else:
                x = np.linspace(subsidy_size[0], subsidy_size[-1], 10)
            #output
            y = []
            for val in x:
                y.append(self._annualized_return(bid_rate=bid_rate, project_size=project_size, loan_size=loan_size, pay_emi_in=pay_emi_in, subsidy_size=val, realized=realized, DCR_status=DCR_status, cost_per_bigha_per_month = cost_per_bigha_per_month, monthly_expenses=monthly_expenses, raise_rate=raise_rate))
            plt.plot(x, y)
            plt.scatter(x, y)
            plt.grid(True)
            plt.xlabel('Subsidy Size')
            if realized:
                plt.ylabel(f'Realized Return over {round(self.INFLATION*100, 2)}% inflation')
            else:
                plt.ylabel('Nominal Return')
        else:
            print('Error! Get better idiot (Check Spelling).')
        return
    
    def full_roi(self, bid_rate: float, project_size: float, loan_size: float, pay_emi_in: int, subsidy_size: float, realized=True, DCR_status=True, cost_per_bigha_per_month = 3e4, monthly_expenses=5e4, raise_rate=1/2, _output=False):
        real_amount = list(self.real_amount(bid_rate=bid_rate, project_size=project_size, loan_size=loan_size, pay_emi_in=pay_emi_in, subsidy_size=subsidy_size, DCR_status=DCR_status, cost_per_bigha_per_month = cost_per_bigha_per_month, monthly_expenses=monthly_expenses, raise_rate=raise_rate)['Real Amount'])
        if DCR_status:
            overall_investment = (project_size * self.DCR) - self.loan_amount(project_size=project_size, subsidy_size=subsidy_size, DCR_status=DCR_status, loan_size=loan_size) - subsidy_size
        else:
            overall_investment = (project_size * self.NON_DCR) - self.loan_amount(project_size=project_size, subsidy_size=subsidy_size, DCR_status=DCR_status, loan_size=loan_size) - subsidy_size
        print(f'Investment of {round(overall_investment, 0)} INR.')
        total_real_return = 0
        for i in range(len(real_amount)):
            total_real_return += real_amount[i]
            if total_real_return >= overall_investment:
                diff = overall_investment - (total_real_return - real_amount[i])
                ratio_to_add = diff/real_amount[i]
                year = i-1
                break
        if _output:
            return round(year+1+ratio_to_add, 1)
        else:
            print(f'{round(year+1+ratio_to_add, 1)} years to get a full ROI.')
        
        return

    def compare(self, bid_rate: float, project_size: float, loan_size: float, pay_emi_in: int, subsidy_size: float, compare: str, realized=True, DCR_status=True, cost_per_bigha_per_month = 3e4, monthly_expenses=5e4, raise_rate=1/2):
        if compare == 'DCR_status':
            print('DCR Case')
            self.annualized_return(bid_rate=bid_rate, project_size=project_size, loan_size=loan_size, pay_emi_in=pay_emi_in, subsidy_size=subsidy_size, realized=realized, DCR_status=True, cost_per_bigha_per_month = cost_per_bigha_per_month, monthly_expenses=monthly_expenses, raise_rate=raise_rate)
            print('-'*10)
            print('Non-DCR Case')
            self.annualized_return(bid_rate=bid_rate, project_size=project_size, loan_size=loan_size, pay_emi_in=pay_emi_in, subsidy_size=subsidy_size, realized=realized, DCR_status=False, cost_per_bigha_per_month = cost_per_bigha_per_month, monthly_expenses=monthly_expenses, raise_rate=raise_rate)
        elif compare == 'realized':
            print('Nominal Return')
            self.annualized_return(bid_rate=bid_rate, project_size=project_size, loan_size=loan_size, pay_emi_in=pay_emi_in, subsidy_size=subsidy_size, realized=False, DCR_status=DCR_status, cost_per_bigha_per_month = cost_per_bigha_per_month, monthly_expenses=monthly_expenses, raise_rate=raise_rate)
            print('-'*10)
            print('Realized Return')
            self.annualized_return(bid_rate=bid_rate, project_size=project_size, loan_size=loan_size, pay_emi_in=pay_emi_in, subsidy_size=subsidy_size, realized=True, DCR_status=DCR_status, cost_per_bigha_per_month = cost_per_bigha_per_month, monthly_expenses=monthly_expenses, raise_rate=raise_rate)
        else:
            self._figure_return(bid_rate=bid_rate, project_size=project_size, loan_size=loan_size, pay_emi_in=pay_emi_in, subsidy_size=subsidy_size, compare=compare, realized=realized, DCR_status=DCR_status, cost_per_bigha_per_month = cost_per_bigha_per_month, monthly_expenses=monthly_expenses, raise_rate=raise_rate)
        return
    def _overall_investment(self, project_size: float, loan_size: float, subsidy_size: float, DCR_status=True):
        if DCR_status:
            return (project_size * self.DCR) - self.loan_amount(project_size=project_size, subsidy_size=subsidy_size, DCR_status=DCR_status, loan_size=loan_size) - subsidy_size
        else:
            return (project_size * self.NON_DCR) - self.loan_amount(project_size=project_size, subsidy_size=subsidy_size, DCR_status=DCR_status, loan_size=loan_size) - subsidy_size
    
    def land_need(self, project_size: float):
        return 4.0*project_size #bighas of land
    
    def land_cost(self, project_size: float, cost_per_bigha_per_month = 3e4):
        total_land_cost = [cost_per_bigha_per_month] * self.YOJANA_LENGTH
        for i in range(0, len(total_land_cost), 2):
            total_land_cost[i] *= (1 + self.LAND_INCREASE)**(i//2)
            if i + 1 < len(total_land_cost):
                total_land_cost[i+1] *= (1 + self.LAND_INCREASE)**(i//2)

        dictionary = {'Year': np.arange(1, 26), 'Land Cost': total_land_cost}
        land_cost_by_year = pd.DataFrame.from_dict(dictionary)
        land_cost_by_year['Land Cost'] *= self.land_need(project_size=project_size)

        return land_cost_by_year
    
    def expenses(self, monthly_expenses=5e4, raise_rate=1/2):
        total_expenses = [monthly_expenses] * self.YOJANA_LENGTH 
        for i in range(len(total_expenses)):
            total_expenses[i] *= (1 + (self.INFLATION)*raise_rate)**(i)
        dictionary = {'Year': np.arange(1, 26), 'Expense Cost': total_expenses}
        total_expenses_by_year = pd.DataFrame.from_dict(dictionary)
        total_expenses_by_year['Expense Cost'] *= 12
        
        return total_expenses_by_year
    
    def sigmoid(self, x):
        return 1.0 / (1.0 + np.exp(-x))
    
    def softplus(self, x):
        return np.log(1 + np.exp(x))
    
    def smoothing(self, x):
        return 0.1 if x < 0
    
    #def score(self, bid_rate: float, project_size: float, loan_size: float, pay_emi_in: int, subsidy_size: float, realized=True, DCR_status=True, cost_per_bigha_per_month = 3e4, monthly_expenses=5e4, raise_rate=1/2)
    

    def generate_latex_report(self, bid_rate: float, project_size: float, loan_size: float, pay_emi_in: int, subsidy_size: float, realized=True, DCR_status=True, cost_per_bigha_per_month = 3e4, monthly_expenses=5e4, raise_rate=1/2):
        # Generate image
        plt.figure(figsize=(15, 6))
        self.figure_total(bid_rate, project_size, loan_size, pay_emi_in, subsidy_size, realized, DCR_status)
        plt.savefig("figure_total.png")
        
        # Get full ROI
        full_roi_output = self.full_roi(bid_rate, project_size, loan_size, pay_emi_in, subsidy_size, realized, DCR_status, cost_per_bigha_per_month = cost_per_bigha_per_month, monthly_expenses=monthly_expenses, raise_rate=raise_rate, _output=True)
        overall_investment = self._overall_investment(project_size=project_size, loan_size=loan_size, subsidy_size=subsidy_size, DCR_status=DCR_status)
        # Add data to the table
        nominal_return = self.nominal_amount(bid_rate, project_size, loan_size, pay_emi_in, subsidy_size, DCR_status,cost_per_bigha_per_month = cost_per_bigha_per_month, monthly_expenses=monthly_expenses, raise_rate=raise_rate)["Nominal Amount"]
        realized_return = self.real_amount(bid_rate, project_size, loan_size, pay_emi_in, subsidy_size, DCR_status, cost_per_bigha_per_month = cost_per_bigha_per_month, monthly_expenses=monthly_expenses, raise_rate=raise_rate)["Real Amount"]
        land_cost = self.land_cost(project_size=project_size, cost_per_bigha_per_month=cost_per_bigha_per_month)['Land Cost']
        land_needed = self.land_need(project_size=project_size)
        expense_cost = self.expenses(monthly_expenses=monthly_expenses, raise_rate=raise_rate)['Expense Cost']

        if DCR_status:
            DCR_Cost = self.DCR
        else:
            DCR_Cost = self.NON_DCR

        overall_nom = nominal_return.sum()
        overall_real = realized_return.sum()


        real_return = self._annualized_return(bid_rate=bid_rate, project_size=project_size, loan_size=loan_size, pay_emi_in=pay_emi_in, subsidy_size=subsidy_size, realized=True, DCR_status=DCR_status, cost_per_bigha_per_month = cost_per_bigha_per_month)
        nom_return = self._annualized_return(bid_rate=bid_rate, project_size=project_size, loan_size=loan_size, pay_emi_in=pay_emi_in, subsidy_size=subsidy_size, realized=False, DCR_status=DCR_status, cost_per_bigha_per_month = cost_per_bigha_per_month)
        # LaTeX document content
        latex_content = f"""
        \\documentclass[10pt]{{article}}
        \\usepackage[left=0.5cm,right=0.5cm,top=0.5cm,bottom=0.5cm]{{geometry}}
        \\usepackage{{graphicx}}
        \\usepackage{{multirow}}
        \\usepackage{{multicol}}
        \\usepackage{{tfrupee}}
        \\begin{{document}}

        % Title
        \\title{{\\fontsize{{20}}{{24}}\\bfseries\\underline{{Solar Report}}}}
        \\author{{}}
        \\date{{}}
        \\maketitle
        \\vspace{{-0.5cm}} % Add some vertical space

        % Add line with project details
        \\vspace{{-1cm}}
        \\begin{{center}}
        \\textbf{{Bid Rate:}} \\rupee~{bid_rate}, \\textbf{{Project Size:}} {project_size}MW, \\textbf{{Loan Size:}} {loan_size}, \\textbf{{Subsidy Size:}} \\rupee~{subsidy_size}, \\textbf{{EMI Length:}} {pay_emi_in} years, \\textbf{{Land Needed:}} {land_needed} bighas, \\textbf{{Inflation:}} {round(self.INFLATION*100, 2)}\\%, \\textbf{{Salary Raise:}} {round(self.INFLATION*raise_rate*100, 2)}\\%, \\textbf{{DCR Cost:}} \\rupee~{round(DCR_Cost, 0)}/MW\\\\
        \\vspace{{-0.75cm}} % Add some vertical space
        \\end{{center}}


        % Include image
        \\begin{{figure}}[!htb]
        \\centering
        \\includegraphics[width=\\textwidth]{{figure_total.png}}
        \\caption{{Total Returns over 25 years}}
        \\end{{figure}}
        
        % Start multicol for table and full ROI output
        \\begin{{multicols}}{{2}}
        
        % Table
        \\begin{{tabular}}{{|c|c|c|c|c|}}
        \\hline
        Year & Nominal & Realized & Land & Expense\\\\
        \\hline
        """
        # Add data to the table
        for year, nominal, realized, land, expense in zip(range(1, len(nominal_return) + 1), nominal_return, realized_return, land_cost, expense_cost):
            latex_content += f"{year} & \\rupee~{nominal:.0f} & \\rupee~{realized:.0f} & \\rupee~{land:.0f} & \\rupee~{expense:.0f} \\\\ \n"

        # Complete LaTeX content for the table
        latex_content += """
        \\hline
        \\end{tabular}
        
        % End first column with a line
        \\columnbreak
        
        % Key Facts title
        \\textbf{\\textit{Key Facts}}:
        
        % Full ROI output as bullet point
        \\begin{itemize}
        """
        # Add full ROI output as a bullet point
        latex_content += f"\\item\\textbf{{Overall Investment}}: \\rupee~{round(overall_investment, 0)}"
        latex_content += f"\\item\\textbf{{Overall Nominal Return}}: \\rupee~{round(overall_nom, 0)}"
        latex_content += f"\\item\\textbf{{Overall Real Return}}: \\rupee~{round(overall_real, 0)} over {round(self.INFLATION*100, 2)}\\% inflation"
        latex_content += f"\\newline"
        latex_content += f"\\item\\textbf{{Nominal Returns}}: {round(nom_return, 2)}\\% returns over 25 years"
        latex_content += f"\\item\\textbf{{Real Returns}}: {round(real_return, 2)}\\% returns over {round(self.INFLATION*100, 2)}\\% inflation over 25 years"
        latex_content += f"\\newline"
        latex_content += f"\\item\\textbf{{Full ROI In}}: {round(full_roi_output, 1)} years"
        # End multicol
        latex_content += "\\end{itemize}\\end{multicols}"
        
        # Complete LaTeX content
        latex_content += "\\end{document}"

        # Write content to a .tex file
        with open("report.tex", "w") as f:
            f.write(latex_content)

        # Compile LaTeX to PDF
        subprocess.run(["pdflatex", "report.tex"])

        print("PDF report generated successfully.")

In [7]:
PMYojana()._annualized_return(bid_rate=3.19, project_size=3.2, loan_size=7e7, pay_emi_in=15, subsidy_size=1.07e7, realized=True, DCR_status=False)

15.44912694463112

In [77]:
#training phase
bid_rate_range = np.random.uniform(2.35, 3.65, (1000,))
project_size_range = np.random.uniform(1., 6.5, (1000,))
loan_size_range = np.random.uniform(60, 80, (1000,))
emi_size_range = np.random.choice([val for val in range(7, 16)], (1000,))
subsidy_size_range = np.random.uniform(25, 95, (1000,))
DCR_status_range = np.random.choice([0,1], 1000)


In [ ]:
#pick up random cases
case_try = set()
for _ in range(30):
    case_try.add(np.random.randint(1, 1000))
case_try

In [87]:
store = []

for val in case_try:
    ret = PMYojana()._annualized_return(bid_rate=bid_rate_range[val], project_size=project_size_range[val], loan_size=loan_size_range[val], pay_emi_in=emi_size_range[val], subsidy_size=subsidy_size_range[val], realized=True, DCR_status=DCR_status_range[val])
    print(ret, PMYojana().sigmoid(PMYojana().softplus(ret - 5)))

3.3305007628161354 0.5430329015688925
3.7586289395711514 0.5631256610083677
3.4697569399762873 0.5488348068520583
3.2675929871507803 0.5406223513994156
6.909834847943608 0.8857400575877833
3.114227107113643 0.5352537361616994
3.7942671233136105 0.5651176411460382
5.050327764203666 0.6723047424259516
0.3376964414995465 0.5023500723952184
1.689803605675566 0.5089636242306961
3.5261046796352336 0.5513746854276561
3.8812504493412447 0.5702045224133878
5.950293107888127 0.7819672842739871
4.221497165979038 0.5933459747093711
3.7264695345661902 0.561373094142949
4.3670180647017 0.6049001878463517
3.8371675112888637 0.5675861713991114
5.153053409623709 0.6840828845119327
5.530779353126336 0.7297484934318589
4.924791256541283 0.6584174300864071
4.441749090371139 0.6112291082756419
2.0981325947686535 0.5133631844982377
2.6175875779002933 0.5220633653703209
2.4670056556251208 0.5190968678818446
7.1539160354605835 0.9058251204864142
-0.912555264587056 0.5006754027729278
5.799660530236994 0.763301